## Training file paired with LambdaAutoTraining project

In [ ]:
import tensorflow as tf
import tensorflowjs as tfjs
from tensorflow import keras
import numpy as np
import boto3
import os
import time
import decimal

**Set environment variables**

In [ ]:
table_models = os.environ.get('TABLE_MODELS')
if not table_models:
    table_models = 'lambdaautotraining-models-dev'
    
table_data = os.environ.get('TABLE_DATA')
if not table_data:
    table_data = 'lambdaautotraining-data-dev'
    
bucket = os.environ.get('BUCKET')
if not bucket:
    bucket = 'lambdaautotraining-dev'
    
region = os.environ.get('REGION')
if not region:
    region = 'us-east-1'
    
print(table_models, table_data, bucket, region)

In [ ]:
dynamodb = boto3.resource('dynamodb', region_name=region)
dynamodb_data = dynamodb.Table(table_data)
dynamodb_models = dynamodb.Table(table_models)

**Retrieve all of the data from DynamoDB**

In [ ]:
response = dynamodb_data.scan()
response_data = response['Items']

while 'LastEvaluatedKey' in response:
    response = dynamodb_data.scan(
        ExclusiveStartKey=response['LastEvaluatedKey']
    )
    response_data += response['Items']

In [ ]:
len(response_data)

**Convert datatypes for training**

In [ ]:
x = []
y = []

In [ ]:
for rec in response_data:
    # convert Decimal datatype to float
    x.append([
        float(rec['temp']),
        float(rec['clo']),
        float(rec['rh']),
        float(rec['wind']),        
    ])
    
    # one-hot encoded
    y_rec = [0,0,0]
    if rec['label'] == 'cold': y_rec[0] = 1
    elif rec['label'] == 'ok': y_rec[1] = 1
    elif rec['label'] == 'warm': y_rec[2] = 1
    y.append(y_rec)

In [ ]:
x_array = np.array(x)
y_array = np.array(y)

**Execute training**

In [ ]:
tf.keras.backend.clear_session()

In [ ]:
model = keras.Sequential()
model.add(keras.layers.Dense(8, input_shape=[4], activation='relu'))
model.add(keras.layers.Dense(3, activation='softmax'))

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(0.1),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x_array, y_array, 
    epochs=25, batch_size=4
)

In [ ]:
model.history.history['loss'][-1:][0]

**Convert model for js and upload**

In [ ]:
created = int(time.time()*1000)
print(created)

In [ ]:
tfjs.converters.save_keras_model(model, './tf_js/')

In [ ]:
for f in os.listdir('./tf_js'):
    boto3.client('s3').upload_file('./tf_js/'+f, bucket, 'models/'+str(created)+'/'+f)
    boto3.client('s3').upload_file('./tf_js/'+f, bucket, 'models/latest/'+f)

**Update model data in dynamodb**

In [ ]:
dynamodb_models.put_item(
    Item={
        'created': created,
        'samples': decimal.Decimal(str(len(response_data))),
        'max_data_created': decimal.Decimal(str(max([int(i['created']) for i in response_data]))),
        'loss': decimal.Decimal(str(model.history.history['loss'][-1:][0])),
        'accuracy': decimal.Decimal(str(model.history.history['acc'][-1:][0])),
    }
)